In [1]:
import pandas as pd
import numpy as np
import requests
import re

In [2]:
df = pd.read_csv('../02_dataCreated/newData20170811.csv')

/home/ram22/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.loc[df['country_of_origin'].isnull(), 'country_of_origin'] = 'MISSING'
df.loc[df['shipper_declared'] == 'NOT DECLARED', 'shipper_unified_id'] = np.nan
df.loc[df['calc_value_usd'] == 0, 'calc_value_usd'] = np.nan 
df.loc[df['calc_weight_kg'] == 0, 'calc_weight_kg'] = np.nan 

df['consignee_comp_size_desc'] = 'Large'
df.loc[df['consignee_d_b_employees_total'] < 100, 'consignee_comp_size_desc'] = 'Medium'
df.loc[df['consignee_d_b_employees_total'] < 19, 'consignee_comp_size_desc'] = 'Small'

df['consignee_comp_size_code'] = 4
df.loc[df['consignee_d_b_employees_total'] < 100, 'consignee_comp_size_code'] = 3
df.loc[df['consignee_d_b_employees_total'] < 19, 'consignee_comp_size_code'] = 2


In [4]:
listCia = list(df.consignee_d_b_business_name.value_counts(dropna=False).index)

In [5]:
naicsTOisic = pd.read_csv('../02_dataCreated/naics_to_isic.csv')
df = df.drop('_merge', axis=1).merge(
    naicsTOisic, how='left',
    left_on='consignee_d_b_primary_naics_1_1_code',
    right_on='naics_2012',
    indicator=True)

In [6]:
print(df._merge.value_counts())
print(df.consignee_d_b_business_name[df._merge == 'left_only'].value_counts())

both          30327
left_only       220
right_only        0
Name: _merge, dtype: int64
Louis Dreyfus Holding B.V.     219
IBERDROLA, SOCIEDAD ANONIMA      1
Name: consignee_d_b_business_name, dtype: int64


In [7]:
fields = [('consignee_d_b_city_name', 'cityName'),
          ('consignee_d_b_year_started', 'dateEstablished'), 
          ('consignee_d_b_country_name', 'countryName'),
          ('consignee_d_b_primary_naics_1_1_code', 'primaryNAICS'),
          ('consignee_d_b_primary_naics_1_1_description', 'primaryNAICSDescription'),
          ('consignee_d_b_sales_volume_us_dollars', 'salesVolume'),
          ('consignee_d_b_employees_total', 'sizeOfOrganization'),
          ('consignee_d_b_latitude', 'latitude'),
          ('consignee_d_b_longitude', 'longitude'),
          ('consignee_d_b_state_province_name', 'stateProvinceName'),
          ('consignee_d_b_street_address', 'streetAddress'),
          ('isic_rev4_2dig', 'industryISICCategory'),
          ('consignee_d_b_url_domain_name_1', 'url'),
          ('consignee_comp_size_desc', 'sizeOfOrganizationType'),
          ('consignee_comp_size_code', 'sizeOfOrganizationTypeDropDownListId'),
          ('import_upstream', 'importsUpstreamness'),
          ('gap_upstream', 'gapUpstreamness')]

In [8]:
upstrDF = pd.read_csv('../02_dataCreated/HS2_upstreamness.csv')
upstrDict = dict(zip(upstrDF['hs2'], upstrDF['upstreamness']))

hsRaw = df['hs_code'].copy(deep=True)
hsRaw = hsRaw.apply(lambda x: list(re.split(r'\s*,\s*',
                 x.replace('-', ',').replace(
                     'nov', '11').replace('ago', '08').replace('ND', '00').replace('EC', '00')))) 
upstream = []
upweightsNum = []
upweightsDen = []
for i in hsRaw:
    holderCode = []
    for j in i:
        if int(j) in upstrDict: 
            holderCode.append(upstrDict[int(j)])
        else:
            holderCode.append(np.nan)
    upstream.append(np.array(holderCode))
    upweightsNum.append(len(holderCode))
    upweightsDen.append(len(holderCode) - np.isnan(holderCode).sum())
upstream = pd.Series(upstream)
upweightsNum = pd.Series(upweightsNum)
upweightsDen = pd.Series(upweightsDen)

df['numerator'] = upstream.multiply(df['calc_value_usd']).divide(upweightsNum).apply(sum) 
df['denominator'] = df['calc_value_usd'].multiply(upweightsDen).divide(upweightsNum) 
byCia = df[['calc_value_usd', 'numerator', 'denominator', 'consignee_d_b_business_name']].groupby('consignee_d_b_business_name')
upstreamness = (byCia['numerator'].sum()) / (byCia['denominator'].sum())
df = df.drop(['numerator', 'denominator', '_merge'], axis=1)

df = df.merge(pd.DataFrame(upstreamness, columns=['import_upstream']).reset_index(), how='inner', left_on='consignee_d_b_business_name', right_on='consignee_d_b_business_name', indicator=True)

df['gap_upstream'] = 'None'

In [9]:
def listCompare(varPair = None):
    print()
    print(varPair)
    temp = cia[[varPair[0], 'country_of_origin']][cia[varPair[0]].notnull()]
    byCountry = temp.groupby('country_of_origin')
    if varPair[3] == sum:
        countryList = byCountry[varPair[0]].apply(varPair[3])
    else:
        countryList = byCountry[varPair[0]].unique().apply(varPair[3])
    if len(countryList) != len(ciaDict[each][varPair[1]]):
        print('In API, not in mine:')
        print(pd.Series(ciaDict[each][varPair[1]]).index.difference(pd.Series(countryList).index))
        print('In mine, not in API:')
        print(pd.Series(countryList).index.difference(pd.Series(ciaDict[each][varPair[1]]).index))

    countryUnion = list(countryList.index.intersection(pd.Series(ciaDict[each][varPair[1]]).index))
    countryDic = dict(countryList)
    for country in countryUnion:
        if round(countryDic[country], 2) != round(ciaDict[each][varPair[1]][country], 2):
            print(country + ': ' + str(countryDic[country]) + ', ' + str(ciaDict[each][varPair[1]][country]))
    print()
    
    if round(countryList.sum(), 2) != round(ciaDict[each][varPair[2]], 2):
        print(varPair[0] + ': ' + str(countryList.sum()) + ', ' + str(ciaDict[each][varPair[2]]) + '\n')

In [10]:
ciaApi = []
url = 'http://staging-meshintelapi.herokuapp.com/organizations'
ciaApi = requests.get(url).json()
ciaDict = {}
for item in ciaApi:
    ciaDict[item['name']] = item

In [11]:
for each in listCia:
   
    cia = df[df['consignee_d_b_business_name'] == each]
    
    print(each)
    for f in fields:
        try:
            if (cia[f[0]].unique()[0] != ciaDict[each][f[1]]):
                try: 
                    if cia[f[0]].unique()[0] != float(ciaDict[each][f[1]]):
                        print(f[0] + ': ' + str(cia[f[0]].unique()[0]) + ', ' + f[1] + ': ' + str(ciaDict[each][f[1]]))
                except ValueError:
                    if cia[f[0]].unique()[0] != ciaDict[each][f[1]]:
                        print(f[0] + ': ' + str(cia[f[0]].unique()[0]) + ', ' + f[1] + ': ' + str(ciaDict[each][f[1]]))
        except KeyError:
            print('KEY ERROR: ', f[1] + '; ' + f[0] + ' = ' + str(cia[f[0]].unique()[0]))
    # Total over countries
    #temp = cia[['shipper_unified_id', 'country_of_origin']][cia.shipper_unified_id.notnull()]
    #print(len(temp['shipper_unified_id'].unique()))  

    listCompare(['shipper_unified_id', 'topCountriesSuppliedFromNumberOfSuppliers', 'numberOfSuppliers', len])
    listCompare(['calc_value_usd', 'topCountriesSuppliedFromValue', 'totalValueBought', sum])
    listCompare(['calc_weight_kg', 'topCountriesSuppliedFromVolume', 'volumeBought', sum])



Nestlé S.A.
KEY ERROR:  latitude; consignee_d_b_latitude = 0.0
KEY ERROR:  longitude; consignee_d_b_longitude = 0.0
import_upstream: 1.76281767793, importsUpstreamness: 1.48

['shipper_unified_id', 'topCountriesSuppliedFromNumberOfSuppliers', 'numberOfSuppliers', <built-in function len>]


['calc_value_usd', 'topCountriesSuppliedFromValue', 'totalValueBought', <built-in function sum>]


['calc_weight_kg', 'topCountriesSuppliedFromVolume', 'volumeBought', <built-in function sum>]

Wal-Mart Stores, Inc.
import_upstream: 1.45020405014, importsUpstreamness: 1.37

['shipper_unified_id', 'topCountriesSuppliedFromNumberOfSuppliers', 'numberOfSuppliers', <built-in function len>]


['calc_value_usd', 'topCountriesSuppliedFromValue', 'totalValueBought', <built-in function sum>]


['calc_weight_kg', 'topCountriesSuppliedFromVolume', 'volumeBought', <built-in function sum>]

Microsoft Corporation
import_upstream: 1.33896738049, importsUpstreamness: 1.14

['shipper_unified_id', 'topCountriesSupplie

In [50]:
 f = {'import_upstream':['min', 'max', 'mean']}
df[['import_upstream', 'consignee_d_b_business_name']].groupby('consignee_d_b_business_name').agg(f)

In [66]:
df[['import_upstream', 'consignee_d_b_business_name']].drop_duplicates()


,import_upstream,consignee_d_b_business_name
0,1.450204,"Wal-Mart Stores, Inc."
8934,1.762818,Nestlé S.A.
19843,1.843317,"Henry Schein, Inc."
20667,1.126219,PUMA SE
22662,1.338967,Microsoft Corporation
26548,2.380684,MITSUBISHI CORPORATION
27878,3.058667,E. I. Du Pont De Nemours and Company
29403,1.940842,The Coca-Cola Company
30203,1.287646,"Nike, Inc."
30246,1.759054,Newmont Mining Corporation


In [52]:
df.shape

(30547, 368)

In [29]:
listCia

['Nestlé S.A.',
 'Wal-Mart Stores, Inc.',
 'Microsoft Corporation',
 'PUMA SE',
 'E. I. Du Pont De Nemours and Company',
 'MITSUBISHI CORPORATION',
 'Henry Schein, Inc.',
 'The Coca-Cola Company',
 'Louis Dreyfus Holding B.V.',
 'BAE SYSTEMS PLC',
 'Nike, Inc.',
 'Newmont Mining Corporation',
 'Qiagen N.V.',
 'UNILEVER PLC',
 'Qualcomm Incorporated',
 'IBERDROLA, SOCIEDAD ANONIMA']

In [85]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(list(x))
    pd.reset_option('display.max_rows')

In [106]:
df['import_upstream'].describe()
    

count    30543.000000
mean         1.671756
std          0.414535
min          1.126219
25%          1.450204
50%          1.762818
75%          1.762818
max          3.058667
Name: import_upstream, dtype: float64

In [170]:
testCorr = df[['import_upstream', 'consignee_d_b_business_name']].groupby('consignee_d_b_business_name').max()
name = []
up = []
for i in ciaDict.keys():
    try:
        up.append(ciaDict[i]['importsUpstreamness'])
        name.append(ciaDict[i]['name'])
    except:
        pass
fromAPI = pd.DataFrame({'name':name, 'up':up})
testCorr.merge(fromAPI, how='inner', left_index=True, right_on='name')[['import_upstream', 'up']].corr()

,import_upstream,up
import_upstream,1.000000,0.744618
up,0.744618,1.000000


In [146]:
name = []
up = []
ciaDict['Wal-Mart Stores, Inc.'].keys()
#ciaDict['Wal-Mart Stores, Inc.']['name'] 
#ciaDict['Wal-Mart Stores, Inc.']['importsUpstreamness'] 


1945

In [148]:
len(ciaDict)

23

In [203]:
test = df['hs_code'].copy(deep=True)
test = test.apply(lambda x: list(re.split(r'\s*,\s*',
                 x.replace('-', ',').replace(
                     'nov', '11').replace('ago', '08').replace('ND', '00').replace('EC', '00')))) 
def fun(x):
    hold = []
    for i in x:
        hold.append(len(str(i)))
    return max(hold)

maxLenHS = test.apply(fun)
"""for i in zip(test.head(20), maxLenHS.head(20)):
    print(i)
"""
print(maxLenHS.max())

another = pd.Series([['08', '4', '10'], ['9', '100'], ['10', '20', '1000'], ['900'], [ '34', '1000000']])
print(another.apply(fun))

2
0    2
1    3
2    4
3    3
4    7
dtype: int64


In [228]:
order = testCorr.merge(fromAPI, how='inner', left_index=True, right_on='name').sort('up')

/home/ram22/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


In [230]:
order.index = range(len(order))
order

,import_upstream,name,up
0,1.450204,"Wal-Mart Stores, Inc.",0.53
1,1.126219,PUMA SE,1.09
2,1.338967,Microsoft Corporation,1.14
3,1.762818,Nestlé S.A.,1.17
4,1.287646,"Nike, Inc.",1.25
5,1.525358,Louis Dreyfus Holding B.V.,1.28
6,1.843317,"Henry Schein, Inc.",1.57
7,1.675351,BAE SYSTEMS PLC,1.59
8,2.009634,Qualcomm Incorporated,1.59
9,1.802638,UNILEVER PLC,1.69


In [236]:
order.sort('import_upstream').rename(columns={'up': 'API_upstream', 'import_upstream' : 'HS2'})

/home/ram22/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


,HS2,name,API_upstream
1,1.126219,PUMA SE,1.09
4,1.287646,"Nike, Inc.",1.25
2,1.338967,Microsoft Corporation,1.14
0,1.450204,"Wal-Mart Stores, Inc.",0.53
5,1.525358,Louis Dreyfus Holding B.V.,1.28
13,1.644184,"IBERDROLA, SOCIEDAD ANONIMA",2.71
7,1.675351,BAE SYSTEMS PLC,1.59
10,1.759054,Newmont Mining Corporation,1.79
3,1.762818,Nestlé S.A.,1.17
9,1.802638,UNILEVER PLC,1.69
